(always be aware of your imports and <b><u><i>preserve namespaces</i></u></b>!!!)

In [1]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

%matplotlib tk

plt.ion()
plt.rcParams["image.cmap"] = "gist_gray"

---

## Advanced (and relational) indexing: Part 2

Last time we saw indexing based on position within an image, we can index on pixel <i>values</i> which is useful for selecting subcomponents of an image that are not conected compoenents.  As an example, let's create an image consisting of a blue circle and line:

In [2]:
rind = np.arange(20000).reshape(200, 100) // 100
cind = np.arange(20000).reshape(200, 100) % 100
two_ln = ((rind > 130) & (rind < 150)) | \
    (np.sqrt((rind - 50)**2+(cind - 50)**2) < 20)
two_ln_c = np.dstack([64 * two_ln, 128 * two_ln, 
                      192 * two_ln]).astype(np.uint8)

Plotting it:

In [3]:
fig0, ax0 = plt.subplots(num=0, figsize=[3, 5])
im0 = ax0.imshow(two_ln_c)
fig0.canvas.draw()

Now identify the pixels in 2D which have the specified color and set those pixels to brown,

In [4]:
index = (two_ln_c[..., 0] == 64) & \
    (two_ln_c[..., 1] == 128) & \
    (two_ln_c[..., 2] == 192)
two_ln_c[index] = [192, 128, 64]

and replot.

In [5]:
im0.set_data(two_ln_c)
fig0.canvas.draw()

In [6]:
plt.close(0)

Now let's try it on an actual image.

In [7]:
dpath  = "images"
fname  = "city_image.jpg"
infile = os.path.join(dpath, fname)
img    = nd.imread(infile)

nrow, ncol = img.shape[:2]
xsize = 8.
ysize = xsize * float(nrow) / float(ncol)

fig0, ax0 = plt.subplots(num=0, figsize=[xsize, ysize])
fig0.subplots_adjust(0, 0, 1, 1)
ax0.grid("off")
im0 = ax0.imshow(img)
fig0.canvas.draw()

#### TODO: Highlight predominanly blue pixels

In [8]:
ind_blu = img.argmax(-1) == 2
im0.set_data(ind_blu)
im0.set_clim((0, 1))
fig0.canvas.draw()

In [9]:
hl = (img[ind_blu] * np.array([0.1, 1.0, 0.1])) \
    .astype(np.uint8)
img[ind_blu] = hl
im0.set_data(img)
fig0.canvas.draw()

In [10]:
plt.close(0)

---

## Example: Invasive Species

Freshkills is a 2,200 acre former landfill on Staten Island that is in the process of being converted into a park (Freshkills park).  There is significant contamination by an invasive species called phragmites.

#### CUSP PhD student Nick Johnson flew a balloon over Freshkills Park to image the extent of the phragmites invasion.

In [11]:
dpath  = "images"
dfile  = "phrag-annotated.png"
infile = os.path.join(dpath, dfile)
img    = nd.imread(infile)[:, :, :3] # ignore the alpha channel
nrow, ncol = img.shape[:2]

xsize = 6.5
ysize = xsize * float(nrow) / float(ncol)
fig0, ax0 = plt.subplots(num=0, figsize=[xsize, ysize])
fig0.subplots_adjust(0,0,1,1)
ax0.axis("off")
im0 = ax0.imshow(img)
fig0.canvas.draw()

What separates the phragmites from the healthy vegetation is the <i>color</i>.  But we already know how to index on color, so let's get an idea of what color we're looking for.  If we zoom in on a region dominated by phragmites:

In [12]:
prow = [1460, 1605]
pcol = [4015, 4354]
llp  = (pcol[0], prow[0])
widp = pcol[1] - pcol[0]
hgtp = prow[1] - prow[0]

ax0.add_patch(plt.Rectangle(llp, widp, hgtp, facecolor='none', 
                            edgecolor='orange', lw=2))
ax0.set_xlim(pcol[0] - 100, pcol[1] + 100)
ax0.set_ylim(prow[0] - 100, prow[1] + 100)
fig0.canvas.draw()

We can see that it's a bit bluer than a region with healthy vegetation:

In [13]:
hrow = [2650, 2905]
hcol = [570, 915]
llh  = (hcol[0], hrow[0])
widh = hcol[1] - hcol[0]
hgth = hrow[1] - hrow[0]

ax0.add_patch(plt.Rectangle(llh, widh, hgth, facecolor='none', 
                            edgecolor='orange', lw=2))
ax0.set_xlim(hcol[0] - 100, hcol[1] + 100)
ax0.set_ylim(hrow[0] - 100, hrow[1] + 100)
fig0.canvas.draw()

In [14]:
ax0.set_xlim(0, ncol)
ax0.set_ylim(nrow, 0)
fig0.canvas.draw()

To start understanding what kind of indexing we need to do, let's make some histograms.

In [15]:
stamp_ph = img[prow[0]:prow[1], pcol[0]:pcol[1]]
stamp_he = img[hrow[0]:hrow[1], hcol[0]:hcol[1]]

In [16]:
clrs = ["red", "green", "blue"]
fig1, ax1 = plt.subplots(2, 2, num=1, figsize=[2 * xsize, xsize])

[ax1[0, 0].hist(stamp_ph[:, :, i].flatten(), bins=128, range=[0, 255], 
               normed=True, color=clrs[i]) for i in range(3)]
[ax1[1, 0].hist(stamp_he[:, :, i].flatten(), bins=128, range=[0, 255], 
               normed=True, color=clrs[i]) for i in range(3)]
[i.set_yticklabels('') for j in ax1 for i in j]

ax1[0, 0].set_ylabel("phragmites")
ax1[1, 0].set_ylabel("healthy")
fig1.canvas.draw()

So the first thing we can do is cut on the ratio of green to blue since the healthy vegetation has a ratio >1.5 for (more or less) all pixels in our patch.

In [17]:
red, grn, blu = img.transpose(2, 0, 1)

In [20]:
img.transpose(2, 0, 1).shape

(3, 6024, 5453)

In [21]:
hind = (1.0 * grn) / (1.0 * blu) < 1.5

/Users/gdobler/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in divide
  if __name__ == '__main__':
/Users/gdobler/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
/Users/gdobler/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':


In [22]:
hind = (1.0 * grn) / (1.0 * blu + (blu == 0)) < 1.5

In [23]:
hflag = [True]
imgs  = [img, hind]
im0.set_clim(0, 1)

def toggle(event):
    """
    Toggle between original image and phragmites indices.
    """

    # -- if the "n" key is pressed
    if event.key == "n":

        # flip the display flag
        hflag[0] = ~hflag[0]
        
        # reset the data
        im0.set_data(imgs[hflag[0]])
        fig0.canvas.draw()
        
dum = fig0.canvas.mpl_connect("key_press_event", toggle)

Looks like we're getting phragmites, dirt, and road.  Let's look at the histograms of the dirt and road:

In [24]:
drow = [2450, 2880]
dcol = [1905, 2235]
rrow = [1665, 1790]
rcol = [3600, 3725]

stamp_di = img[drow[0]:drow[1], dcol[0]:dcol[1]]
stamp_rd = img[rrow[0]:rrow[1], rcol[0]:rcol[1]]

for reg in [[drow, dcol], [rrow, rcol]]:
    ax0.add_patch(plt.Rectangle((reg[1][0], reg[0][0]), 
                                reg[1][1] - reg[1][0], 
                                reg[0][1] - reg[0][0],
                  facecolor='none', edgecolor='orange', lw=2))
fig0.canvas.draw()

In [25]:
[ax1[0, 1].hist(stamp_di[:, :, i].flatten(), bins=128, range=[0, 255], 
               normed=True, color=clrs[i]) for i in range(3)]
[ax1[1, 1].hist(stamp_rd[:, :, i].flatten(), bins=128, range=[0, 255], 
               normed=True, color=clrs[i]) for i in range(3)]

ax1[0, 1].set_ylabel("dirt")
ax1[1, 1].set_ylabel("road")

fig1.canvas.draw()

It looks like "dirt" has a high red-to-blue ratio compared to phragmites, so we update our definition of phragmites colors:

In [26]:
hind = ((1.0 * grn) / (1.0 * blu + (blu == 0)) < 1.5) & \
    ((1.0 * red) / (1.0 * blu + (blu == 0)) < 1.2)

imgs = [img, hind] # reset for toggle function

Lastly, the road mostly has values $>200$ while the phragmites are $<200$ (also let's use the fact that phragmites have red values which are $<150$ and the lines are black).

In [27]:
hind = ((1.0 * grn)/(1.0 * blu + (blu == 0)) < 1.5) & \
    ((1.0 * red) / (1.0 * blu + (blu == 0)) < 1.2) & \
    (img.max(2) < 200) & \
    (red < 150) & \
    (img.max(2) > 50)

imgs = [img, hind] # reset for toggle function

In [25]:
plt.close("all")